# Load the saved model

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer
from tensorflow.keras.models import load_model
import numpy as np
import joblib

from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.keras.layers import Layer

class BertLayer(Layer):
    def __init__(self, **kwargs):
        super(BertLayer, self).__init__(**kwargs)
        self.bert = TFBertModel.from_pretrained("bert-base-uncased")

    def call(self, inputs):
        input_ids, attention_mask = inputs
        return self.bert(input_ids=input_ids, attention_mask=attention_mask)[0]

    def get_config(self):
        config = super(BertLayer, self).get_config()
        return config

In [ ]:
from tensorflow.keras.models import load_model
from transformers import BertTokenizer, TFBertModel

model = load_model('/content/drive/MyDrive/BugTriage/models/bug_triage_model_v1.h5', custom_objects={'BertLayer': BertLayer})


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

# Load the tokenizer

In [ ]:
# Load the tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/BugTriage/models/tokenizer_v1')

# Load the label encoder

In [ ]:
# Load the label encoder
target_label_encoder = joblib.load('/content/drive/MyDrive/BugTriage/models/label_encoder_v1.pkl')

# Predict developer function

In [ ]:
def predict_developer(bug_description, severity, priority):
    """Predicts the most suitable developer based on bug description, severity, and priority."""
    # Tokenize the description
    new_bug_tokens = bert_tokenizer([bug_description], padding='max_length', truncation=True, return_tensors='tf', max_length=128)

    # Prepare input
    prediction_inputs = {
        'input_ids': new_bug_tokens['input_ids'],
        'attention_mask': new_bug_tokens['attention_mask'],
        'severity_input': np.array([[severity]], dtype=np.float32),
        'priority_input': np.array([[priority]], dtype=np.float32)
    }

    # Make prediction
    predicted_probabilities = model.predict(prediction_inputs)
    predicted_developer_id = np.argmax(predicted_probabilities, axis=1)
    predicted_developer = target_label_encoder.inverse_transform(predicted_developer_id)

    return predicted_developer[0]

In [ ]:
predict_developer("Using keyword or space, can't use the Go button", 3, 5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step


'developer_001'

In [ ]:
predict_developer("Can't use Go button with using keyword or space", 3, 5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step


'developer_006'

# Expose API

In [ ]:
!pip install flask fastapi uvicorn

In [ ]:
!pip install nest_asyncio

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
!pip install pyngrok

In [ ]:
!ngrok authtoken 2t7knVIpKddAYvwATfJG22KQYuV_6TqEzBZmBnMnKLpNWYndw

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install flask_cors

# API

In [ ]:
from fastapi import FastAPI, Request, HTTPException
import tensorflow as tf
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware

# Start ngrok tunnel
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post('/predict')
async def predict(request: Request):
    try:
        data = await request.json()

        # extract input
        bug_description = data['bug_description']
        severity = float(data['severity'])
        priority = float(data['priority'])

        predicted_developer = predict_developer(bug_description, severity, priority)
        return {'predicted_developer': predicted_developer}

    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))


if __name__ == '__main__':
    uvicorn.run(app, host='0.0.0.0', port=8000)

INFO:     Started server process [3087]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://8097-34-125-88-215.ngrok-free.app" -> "http://localhost:8000"
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step
INFO:     112.134.166.163:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     2402:d000:8114:310:300c:8262:a2ef:7ec5:0 - "OPTIONS /predict HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
INFO:     2402:d000:8114:310:300c:8262:a2ef:7ec5:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     2402:d000:8114:310:300c:8262:a2ef:7ec5:0 - "OPTIONS /predict HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
INFO:     2402:d000:8114:310:300c:8262:a2ef:7ec5:0 - "POST /predict HTTP/1.1" 200 OK
